# 🚀 **Passo 2: Otimizando Prompts com Bedrock Prompt Optimizer**

## **Aula 2.1: Por Que Otimizar Prompts?**

---

### **Tá, mas o que é otimização de prompt?**

Imagina que você tá tentando explicar algo pra um amigo, mas ele não tá entendendo. Aí você muda as palavras, dá exemplos diferentes, e de repente ele entende perfeitamente! É isso que a gente vai fazer aqui - só que em vez de amigos, são modelos de IA. 😄

**Por que otimização de prompt é importante?**

Diferentes modelos de IA são como pessoas com personalidades diferentes. O que funciona pra um pode não funcionar pra outro. É como tentar falar português com alguém que só entende inglês - você precisa adaptar sua linguagem!

### **O Bedrock Prompt Optimizer**

Pra esse workshop, vamos usar o [Amazon Bedrock Prompt Optimizer](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management-optimize.html), que é tipo um "tradutor automático" pra prompts. Ele:

1. **Analisa** a estrutura e intenção do seu prompt original
2. **Reformata** pra aproveitar melhor as capacidades do modelo alvo
3. **Otimiza** as instruções pra clareza e eficácia
4. **Preserva** a funcionalidade principal enquanto melhora a performance

Isso economiza horas de trabalho manual de engenharia de prompts e produz resultados mais confiáveis. Vamos ver isso em ação!

---

**🖼️ Sugestão de imagem**: Um diagrama mostrando como prompts são otimizados para diferentes modelos

In [ ]:
# ��️ PREPARANDO O AMBIENTE
import boto3
import random
import json
import sys
import pandas as pd
from IPython.display import display

print("✅ Ferramentas importadas! Vamos começar a otimizar!")

### **Recuperando Nosso Progresso**

Vamos carregar nosso dataframe de tracking do Passo 1, que contém informações sobre nosso modelo fonte e os modelos candidatos que vamos avaliar. Esse dataframe vai servir como nosso repositório central pra todas as métricas de avaliação durante o workshop.

In [ ]:
# 📊 CARREGANDO NOSSO TRACKING
evaluation_tracking_file = '../data/evaluation_tracking.csv'
evaluation_tracking = pd.read_csv(evaluation_tracking_file)
display(evaluation_tracking)

print("\n💡 Perfeito! Agora temos nosso plano de avaliação carregado.")

### **Entendendo o Prompt Original**

#### **Analisando Nosso Ponto de Partida**

Antes de otimizar prompts pros nossos modelos candidatos, precisamos entender a estrutura do prompt usado pelo nosso modelo fonte. Esse prompt define:

1. **Como apresentamos** o contexto de entrada pro modelo
2. **Que tarefa específica** estamos pedindo pro modelo fazer
3. **Quaisquer restrições** ou requisitos de formatação

Vamos primeiro preparar nosso dataframe de tracking pra armazenar prompts pra cada modelo, depois examinar o prompt do modelo fonte:

In [ ]:
# �� PREPARANDO PROMPTS PRA CADA MODELO
evaluation_tracking['text_prompt'] = ''
evaluation_tracking['region'] = 'us-east-1'  # Definindo como us-east-1 por padrão
evaluation_tracking['inference_profile'] = 'standard'  # Perfil padrão ou otimizado

print("✅ Colunas preparadas! Agora vamos adicionar o prompt do modelo fonte.")

In [ ]:
# 📝 ADICIONANDO O PROMPT DO MODELO FONTE
raw_prompt = """
First, please read the article below.
{context}
 Now, can you write me an extremely short abstract for it?
"""

evaluation_tracking.loc[evaluation_tracking['model'] == 'source_model', 'text_prompt'] = raw_prompt
display(evaluation_tracking)

print("\n💡 Esse é o prompt original que o modelo fonte usa. Simples e direto!")
print("\n🔍 Vamos analisar o que esse prompt faz:")
print("• Pede pro modelo ler o artigo")
print("• Solicita um resumo extremamente curto")
print("• Usa uma linguagem informal e direta")

### **O Poder do Bedrock Prompt Optimizer**

Agora vamos usar a mágica do Bedrock Prompt Optimizer! É como ter um especialista em comunicação que sabe exatamente como falar com cada modelo de IA.

#### **Como Funciona**

O Prompt Optimizer é tipo um "tradutor inteligente" que:

1. **Entende** o que seu prompt original quer fazer
2. **Adapta** a linguagem pro modelo específico
3. **Mantém** a intenção original
4. **Melhora** a clareza e eficácia

Vamos ver isso em ação com nossos modelos candidatos:

In [ ]:
# �� CONFIGURANDO O BEDROCK CLIENT
bedrock_client = boto3.client('bedrock')

print("✅ Cliente Bedrock configurado! Vamos começar a otimizar.")

In [ ]:
# �� OTIMIZANDO PROMPTS PRA CADA MODELO
def optimize_prompt_for_model(source_prompt, target_model):
    """
    Otimiza um prompt pra um modelo específico usando o Bedrock Prompt Optimizer.
    É como ter um tradutor especializado pra cada modelo!
    """
    
    try:
        # Configurando a requisição de otimização
        optimization_request = {
            'prompt': source_prompt,
            'targetModel': target_model,
            'optimizationType': 'EFFICIENCY',  # Otimizando pra eficiência
            'constraints': {
                'maxTokens': 1000,  # Limite de tokens
                'preserveIntent': True  # Mantém a intenção original
            }
        }
        
        # Chamando o Prompt Optimizer
        response = bedrock_client.optimize_prompt(**optimization_request)
        
        return response['optimizedPrompt']
        
    except Exception as e:
        print(f"⚠️ Erro ao otimizar prompt para {target_model}: {str(e)}")
        # Se der erro, vamos usar uma versão adaptada manualmente
        return adapt_prompt_manually(source_prompt, target_model)

def adapt_prompt_manually(source_prompt, target_model):
    """
    Adapta o prompt manualmente se o otimizador automático falhar.
    É como ter um plano B na manga!
    """
    
    if 'nova' in target_model:
        # Nova prefere instruções mais estruturadas
        return f"""
        TAREFA: Criar um resumo extremamente conciso
        
        ARTIGO:
        {{context}}
        
        INSTRUÇÕES:
        - Leia o artigo acima
        - Crie um resumo muito curto e direto
        - Mantenha apenas as informações essenciais
        """
    elif 'claude' in target_model:
        # Claude prefere linguagem mais natural
        return f"""
        Por favor, leia o seguinte artigo:
        
        {{context}}
        
        Agora, escreva um resumo extremamente breve e conciso deste artigo.
        """
    else:
        # Fallback genérico
        return source_prompt

print("✅ Funções de otimização criadas! Vamos testar.")

In [ ]:
# 🎯 APLICANDO OTIMIZAÇÃO PRA CADA MODELO
print("�� OTIMIZANDO PROMPTS...")
print("=" * 50)

for index, row in evaluation_tracking.iterrows():
    model = row['model']
    
    if model != 'source_model':  # Só otimizar pros candidatos
        print(f"\n�� Otimizando para: {model}")
        
        # Pegando o prompt original
        original_prompt = evaluation_tracking.loc[evaluation_tracking['model'] == 'source_model', 'text_prompt'].iloc[0]
        
        # Otimizando o prompt
        optimized_prompt = optimize_prompt_for_model(original_prompt, model)
        
        # Salvando o prompt otimizado
        evaluation_tracking.loc[index, 'text_prompt'] = optimized_prompt
        
        print(f"✅ Prompt otimizado salvo!")
        print(f"📝 Tamanho original: {len(original_prompt)} caracteres")
        print(f"�� Tamanho otimizado: {len(optimized_prompt)} caracteres")
        
        # Mostrando uma prévia do prompt otimizado
        print(f"\n🔍 Prévia do prompt otimizado:")
        print(optimized_prompt[:200] + "..." if len(optimized_prompt) > 200 else optimized_prompt)

print("\n" + "=" * 50)
print("�� OTIMIZAÇÃO CONCLUÍDA!")

### **Comparando os Prompts**

Agora vamos dar uma olhada em como os prompts ficaram diferentes pra cada modelo. É como ver como a mesma história é contada de formas diferentes pra públicos diferentes!

In [ ]:
# 📊 COMPARANDO OS PROMPTS OTIMIZADOS
print("�� COMPARAÇÃO DOS PROMPTS:")
print("=" * 60)

for index, row in evaluation_tracking.iterrows():
    model = row['model']
    prompt = row['text_prompt']
    
    print(f"\n🎯 MODELO: {model}")
    print(f"📏 Tamanho: {len(prompt)} caracteres")
    print(f"🔤 Palavras: {len(prompt.split())} palavras")
    print(f"�� Prompt:")
    print("-" * 40)
    print(prompt)
    print("-" * 40)

print("\n�� Observe as diferenças:")
print("• Cada modelo tem um estilo diferente de instrução")
print("• Alguns são mais estruturados, outros mais naturais")
print("• O tamanho varia conforme a complexidade necessária")

### **Salvando Nossos Progressos**

Agora vamos salvar nosso dataframe atualizado com todos os prompts otimizados. Isso vai ser crucial pros próximos passos!

In [ ]:
# 💾 SALVANDO O TRACKING ATUALIZADO
evaluation_tracking.to_csv('../data/evaluation_tracking.csv', index=False)

print("✅ Tracking atualizado salvo!")
print("\n📊 RESUMO DO QUE FIZEMOS:")
print(f"• Modelos avaliados: {len(evaluation_tracking)}")
print(f"• Prompts otimizados: {len(evaluation_tracking[evaluation_tracking['model'] != 'source_model'])}")
print(f"• Arquivo salvo: ../data/evaluation_tracking.csv")

# Mostrando o status final
display(evaluation_tracking[['model', 'text_prompt']])

### **Testando os Prompts Otimizados**

Vamos fazer um teste rápido pra ver se nossos prompts otimizados estão funcionando. É como fazer um "teste de som" antes do show principal!

In [ ]:
# 🧪 TESTE RÁPIDO DOS PROMPTS
def test_prompt_with_sample(prompt, model_id):
    """
    Testa um prompt com uma amostra pequena pra ver se funciona.
    É como fazer um teste de direção antes de comprar o carro!
    """
    
    try:
        # Carregando uma amostra pequena
        sample_data = pd.read_csv('../data/document_sample_2.csv')
        test_document = sample_data.iloc[0]['document']
        
        # Formatando o prompt com o documento
        formatted_prompt = prompt.format(context=test_document)
        
        print(f"\n�� TESTE PARA: {model_id}")
        print(f"�� Prompt formatado (primeiros 200 chars):")
        print(formatted_prompt[:200] + "...")
        print(f"✅ Prompt formatado com sucesso!")
        
        return True
        
    except Exception as e:
        print(f"❌ Erro no teste: {str(e)}")
        return False

# Testando cada prompt
print("🧪 INICIANDO TESTES DOS PROMPTS...")
print("=" * 50)

for index, row in evaluation_tracking.iterrows():
    model = row['model']
    prompt = row['text_prompt']
    
    if prompt:  # Só testar se tem prompt
        success = test_prompt_with_sample(prompt, model)
        if success:
            print(f"✅ {model}: OK!")
        else:
            print(f"❌ {model}: Precisa ajuste!")

print("\n🎉 TESTES CONCLUÍDOS!")

### **Resumo do Passo 2**

�� **Parabéns!** Você acabou de completar o segundo passo da nossa jornada de migração. Vamos recapitular o que fizemos:

✅ **Entendemos a importância**: Diferentes modelos precisam de prompts diferentes
✅ **Usamos o Bedrock Prompt Optimizer**: Ferramenta automática pra otimização
✅ **Otimizamos prompts**: Adaptamos pra cada modelo candidato
✅ **Comparamos resultados**: Vimos as diferenças entre os prompts
✅ **Testamos funcionamento**: Verificamos se tudo tá funcionando
✅ **Salvamos progresso**: Tracking atualizado com prompts otimizados

### **O Que Vem no Próximo Passo**

No próximo notebook, vamos fazer algo super importante: **medir a latência**! É como cronometrar quanto tempo cada modelo leva pra responder. Vamos gerar respostas usando os prompts otimizados que acabamos de criar e coletar métricas detalhadas de performance.

---

**💡 Dica do Pedro**: Otimização de prompt é uma arte! Às vezes pequenas mudanças fazem uma diferença enorme na qualidade das respostas.

**🚀 Próximo passo**: Avaliação de latência dos modelos